### (1) Load Required Packages

In [1]:
from tensorflow.keras.models import model_from_json
#from source.data_processing import process_data
import numpy as np
import pandas as pd
import glob

In [27]:
## Supress warnings
import warnings
warnings.filterwarnings('ignore')

### (2) Load Data

In [2]:
df1 = pd.read_csv('/home/hajar/Dropbox/phd1/CyMET/data/Z2RC.csv', index_col=0).sample(1000)
df1['batch'] = [1]*len(df1)

df2 = pd.read_csv('/home/hajar/Dropbox/phd1/CyMET/data/arthur.csv', index_col=0).sample(1000)
df2['batch'] = [2]*len(df2)

/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### (3) Overlapping markers between studies

In [3]:
## (1) find matching markers across all panels
p1 = list(df1)
p2 = list(df2)
overlap = list(set.intersection(*map(set, [p1, p2])))
print(overlap) #the shared markers between the two studies

['TCRgd', 'CD27', 'CD127', 'CD38', 'CD16', 'CD19', 'HLA-DR', 'CD14', 'CD45RO', 'CD3', 'CD8', 'batch', 'CD25', 'CD11c', 'CCR7', 'CD4', 'CD56', 'CD57', 'sample', 'CD28', 'CD161']


### (4) Prepare data for batch correction 
(requirements for CyCombine include the condition and the batch ID)

In [4]:
df1 = df1[overlap]; df2 = df2[overlap]
frame = pd.concat([df1, df2]).reset_index(drop=True).rename(columns={"sample": "file"}) 

In [5]:
### count the number of healthy and disease samples
hea = []; cov = []; 
for F in frame.file.unique():
    if ('COV' in F) or ('_CV' in F) or ('AP' in F) or ('Covid' in F):
        cov.append(F)
    else:
        hea.append(F)
        
print('number of healthy samples: ',len(hea))
print('number of disease samples: ',len(cov))

number of healthy samples:  44
number of disease samples:  25


In [6]:
cond = []
for F in frame.file:
    if F in cov:
        cond.append('healthy')
    else:
        cond.append('disease')
frame['condition'] = cond

In [7]:
frame = frame.rename(columns={"file": "sample"}) #needs to be renamed for cycombine
frame.sample(5)

,TCRgd,CD27,CD127,CD38,CD16,CD19,HLA-DR,CD14,CD45RO,CD3,...,CD25,CD11c,CCR7,CD4,CD56,CD57,sample,CD28,CD161,condition
428,0.000000,0.000000,0.667143,0.0,0.897032,2.985953,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.187743,0.000000,0.219101,0.000000,CP-01_CyTOF_CD45,0.000000,0.000000,disease
184,0.330026,0.000000,0.967860,0.0,0.000000,0.021406,0.000000,3.513145,0.077981,3.607296,...,0.197574,0.314896,0.000000,0.000000,0.818437,6.194184,HC-04_CyTOF_CD45,0.000000,0.000000,disease
1601,0.000000,2.138342,0.695839,0.0,0.632442,0.000000,0.298995,0.000000,0.695993,1.949126,...,0.000000,0.043175,0.881228,0.000000,0.046441,0.108603,r/200514_Barcode_1_B02,2.007482,0.161986,disease
1105,0.686036,0.000000,0.000000,0.0,2.625585,0.000000,0.327197,0.178850,0.000000,0.282896,...,0.177988,0.449518,0.423015,0.000000,3.252565,4.645685,r/200812_Barcode_10_anchor_F07,0.377723,1.120411,disease
104,0.000000,1.463077,3.393292,0.0,0.000000,0.000000,0.563519,0.429522,2.091100,3.314120,...,0.000000,0.000000,0.715992,0.374919,0.641154,0.000000,HC-01_CyTOF_CD45,0.287985,3.804778,disease


In [8]:
#frame.to_csv('/home/hajar/Dropbox/phd1/CyMET/data/data_uncorrected.csv')

### (5) cyCombine Batch Correction 
Derived from R

In [10]:
%load_ext rpy2.ipython

frame = frame
overlap = overlap

In [11]:
%%R -i frame 
library(cyCombine)
overlap <- c('CD4', 'CD3', 'CD38', 'CD28', 'CD57', 'HLA-DR', 'CD45RO', 
              'CD25', 'CCR7', 'CD11c', 'CD19', 'CD56', 'CD27', 'CD16', 
              'TCRgd', 'CD127', 'CD161', 'CD8', 'CD14')

# import df from global environment
# Batch correct based on overlapping markers
co_corrected <- batch_correct(frame,
                              covar = "condition",
                              xdim = 10,
                              ydim = 10,
                              norm_method = 'rank',
                              markers = overlap)

write.csv(co_corrected, '/home/hajar/Dropbox/phd1/CyMET/data/data_corrected.csv')

R[write to console]: Creating SOM grid..

R[write to console]: Ranking expression data..

R[write to console]: Batch correcting data..

R[write to console]: Correcting Label group 1

R[write to console]: Found2batches

R[write to console]: Adjusting for1covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Correcting Label group 2



Found 3 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


R[write to console]: Found2batches

R[write to console]: Adjusting for1covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Label group 3 only contains cells from batch 1

R[write to console]: Correcting Label group 4



Found 1 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


R[write to console]: Found2batches

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Correcting Label group 5



...

R[write to console]: Found2batches

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Correcting Label group 95

R[write to console]: Found2batches

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Correcting Label group 96

R[write to console]: Using the 'mean only' version of ComBat

R[write to console]: Found2batches

R[write to console]: Note: one batch has only one sample, setting mean.only=TRUE

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: St

Found 2 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


R[write to console]: Found2batches

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Correcting Label group 99



Found 6 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


R[write to console]: Found2batches

R[write to console]: Adjusting for0covariate(s) or covariate level(s)

R[write to console]: Standardizing Data across genes

R[write to console]: Fitting L/S model and finding priors

R[write to console]: Finding parametric adjustments

R[write to console]: Adjusting the Data


R[write to console]: Label group 100 only contains cells from batch 2

R[write to console]: Done!



### (6) Load batch corrected data

In [17]:
df = pd.read_csv('/home/hajar/Dropbox/phd1/CyMET/data/data_corrected.csv').rename(columns={"file": "sample", 
                                                                                           "batch":"study"}) 
df.sample(5)

,Unnamed: 0,id,label,CD4,CD3,CD38,CD28,CD57,HLA-DR,CD45RO,...,CD27,CD16,TCRgd,CD127,CD161,CD8,CD14,study,sample,condition
921,922,922,96,0.049592,0.017947,2.399522,0.036101,1.218071,1.675111,2.169348,...,1.419763,2.716930,2.033725,0.013405,0.002072,0.011974,1.418271,1,AP-02_CyTOF_CD45,healthy
1347,1348,1348,20,0.560716,0.000000,2.972178,0.001193,0.139430,1.656522,0.000000,...,0.288866,4.092747,0.242165,0.615301,2.324675,0.025305,0.509367,2,r/200522_Barcode_2_C04,disease
639,640,640,56,0.021460,0.008685,2.123425,0.449583,0.019407,0.009840,3.107650,...,1.038683,4.445386,3.548361,0.016461,0.000300,0.014579,1.999363,1,AP-03_CyTOF_CD45,healthy
1733,1734,1734,77,0.235333,0.101020,2.462958,1.094629,0.600432,1.185699,1.910532,...,0.000000,2.658660,0.465065,0.000000,0.047210,0.000000,2.647163,2,r/200707_Barcode_8_CV24,healthy
219,220,220,10,0.240160,0.122760,3.458516,0.664939,3.260562,0.179850,0.052589,...,0.000000,4.421995,1.647384,0.024556,1.273243,0.128186,0.000000,1,AP-02_CyTOF_CD45,healthy


### (7) Run meta-prediction
#### Load model

In [18]:
from tensorflow.keras.models import model_from_json #load required packages
json_file = open('/home/hajar/Dropbox/phd1/CyMET/immunopred/model.json', 'r') #open the model
model_tmp = json_file.read() #read the file
json_file.close() #close the file

model = model_from_json(model_tmp) # load model weights
model.load_weights('/home/hajar/Dropbox/phd1/CyMET/immunopred/model_weights.h5')

print("Immunopred Successfully Loaded!")

Immunopred Successfully Loaded!


#### Predict cells

In [29]:
y_pred = predict_cells(df) #gives you the prediction confidence for each cell type

preds = y_pred.argmax(axis=1) #gives you the actual cell type with highest confidence
df['cell_type'] = preds # map the cell type number to name!

#get the confidence score for final prediction:
scores = []
for x,y in zip(df.index, preds):
    sc = y_pred[x][y] #index and then the position of the highest score
    scores.append(sc)
df['score'] = scores

[1, 2]
Predicting:  1 | 1  out of  2
      CD66b is missing - replaced with 0
number of cells in training: 1000
32/32 [==============================] - 0s 706us/step
      done.


Predicting:  2 | 2  out of  2
      CD66b is missing - replaced with 0
number of cells in training: 1000
32/32 [==============================] - 0s 618us/step
      done.


Cells successfully predicted!
